<a href="https://colab.research.google.com/github/orgoca/W207FinalProject/blob/main/Project2EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 2 Baseline
Quora Project



In [143]:
# Code to read csv file from Google Drive into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [144]:
#Import Libraries
import os
import pandas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
import csv

In [145]:
#Load test DF
testLink = 'https://drive.google.com/file/d/1chKVOcXwNzGRe_hRxpGlq18NVaSRM8X9/view?usp=sharing'
idTest = testLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTest}) 
downloaded.GetContentFile('test.csv')  
test = pd.read_csv('test.csv')

In [146]:
#Load train DF
trainLink = 'https://drive.google.com/file/d/1fP-AWiUGLjLkL0c_u67HOUFmHOMO7qm2/view?usp=sharing'
idTrain = trainLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTrain}) 
downloaded.GetContentFile('train.csv')  
train = pd.read_csv('train.csv')

In [147]:
#Split Training Data into MiniTrain and MiniTest (Test data has no labels so impossible to measure error from it)
miniTrain, miniTest = train_test_split(train, test_size=0.2)

In [148]:
#Create a vectorizer, create question and target lists, and vectorize train questions
vectorizer = CountVectorizer()
trainQuestions = miniTrain.question_text.tolist()
trainLabels = miniTrain.target.tolist()
vTrain = vectorizer.fit_transform(trainQuestions)

In [127]:
#What is the vocabulary size, and create a bag of words
print("vocab size: ", vTrain.shape)
bagOfWords = vectorizer.vocabulary_
len(bagOfWords)

vocab size:  (1044897, 172948)


172948

In [152]:
#Create a logistic regression model and fit to train data
lr_clf = LogisticRegression(C=1)
lr_clf.fit(vTrain, trainLabels)

train_predict = lr_clf.predict(vTrain)
train_predict

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0, 0, 0, ..., 0, 0, 0])

In [153]:
lr_clf.predict_proba(vTrain)

array([[0.99280442, 0.00719558],
       [0.9956997 , 0.0043003 ],
       [0.99461185, 0.00538815],
       ...,
       [0.96670189, 0.03329811],
       [0.99870052, 0.00129948],
       [0.7517752 , 0.2482248 ]])

In [222]:
#Try an insincere question. Insincere questions should get a higher probability score to legitimate questions.
testQuestionInsincere1 = "Why are Indians a superior race to Pakistanis?"

vdev = vectorizer.transform([testQuestionInsincere.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.9498919722380983

In [244]:
#Try a sincere question. Sincere questions should get a lower probability score ideally close to zero.
testQuestionInsincere = "What are the origins of the conflict between Indians and Pakistanis?"
vdev = vectorizer.transform([testQuestionInsincere.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.17770639220411022

In [243]:
#Let's predict probability for each question in the miniTest dataframe
columnNames = ['question', 'pred']
predictDF = pd.DataFrame(columns = columnNames)

for question in miniTest.question_text: 
  vdev = vectorizer.transform([question.lower()])
  pred = lr_clf.predict_proba(vdev)[0][1]
  predictDF = predictDF.append({'question':question,'pred': pred}, ignore_index=True)

predictDF

,question,pred
0,Whenever I meditate my body starts to shakes v...,0.054901
1,How can you clear rolling acne scars with home...,0.004890
2,How can I get resent information about all com...,0.005836
3,How do I stop myself from being insulted?,0.020352
4,I want to become a developer both front-end an...,0.004018
...,...,...
261220,"From a security perspective, is a ZTE Z233VL p...",0.000442
261221,How do I start self preparation for CAT 2019?,0.009282
261222,Does coarse hair usually look better with a da...,0.010414
261223,"What does ""exemplary students"" mean?",0.008262


In [267]:
#Let's see some nasty questions
nastyQuestions = predictDF[predictDF.pred > .9]

nastyQuestions.to_csv('nastyQuestions.csv', index=False)

In [268]:
#Merge predict DF to miniTestDF to produce prediction metrics

predictDF.columns = ['question_text', 'pred']

summaryDF = pd.merge(predictDF,miniTest,on='question_text',how='left')

summaryDF.to_csv('summaryDF.csv', index=False)

In [279]:
confusionMatrixDF = summaryDF[['pred', 'target']]

In [280]:
#Tried -by hand- values between 0.5:1.0 and 0.2 produced the best F1 score
 
confusionMatrixDF['pred'] = np.where(confusionMatrixDF['pred'] > 0.2, 1, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [291]:
#Compute baseline F1 Score (and other metrics)
confusionMatrixDF

from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print('Accuracy:', accuracy_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('F1 score:', f1_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Recall:', recall_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Precision:', precision_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('\n clasification report:\n', classification_report(confusionMatrixDF.target,confusionMatrixDF.pred))
print('\n confussion matrix:\n',confusion_matrix(confusionMatrixDF.target, confusionMatrixDF.pred))

Accuracy: 0.9467049478418987
F1 score: 0.6207366241691185
Recall: 0.7071127110228401
Precision: 0.5531656632355797

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97    245113
           1       0.55      0.71      0.62     16112

    accuracy                           0.95    261225
   macro avg       0.77      0.83      0.80    261225
weighted avg       0.95      0.95      0.95    261225


 confussion matrix:
 [[235910   9203]
 [  4719  11393]]
